# Auto-Differentiation

## Short explanation
The idea behind auto-derivation is to construct an evaluation tree for some highly nested function and deriving this function by recursively "evaluating" (where the way to evaluate it is to derive the subtrees) the tree giving the recusion the basic rules of derivation as anchors. By 'basic rules' I mean these: 
<ol>
<li> +: the derivative of any sum is the sum of the derivatives
<li> *: the derivative of any product is the derivative of the first factor times the second plus the first factor times the derivative of the second.
<li> power: $\frac{ \partial x^k}{\partial x} = k \cdot x^{k-1}$
<li> ...
</ol>

## Long excurse

One of the reasons, machine learning could get this powerful is that there is a way to automatically compute very complicated differentials. While it is relatively straightforward to calculate the derivative of the cost-function with respect to the parameters of a single-neuron 'net', calculating the gradient of the weights for a neuron that is not in the output-layer is a bit more involved: the loss is a function of the output-layer-neurons's inputs, (some of) which are functions of the weights in question. The chain of dependency is of course getting longer the deeper and more interconnected a net becomes.
Having to derive all of the gradients ourselves for a big, dense neural net would take forever. 

That is where automatic differentiation comes in. There is a relatively small amount of mathematical operations (+, -, *, /, power, exp, sin, cos) that are the building blocks of any function. Knowing the chain rule, we can calculate arbitrarily nested derivatives. 

Let's call any neuron $\mathcal{N}_{i, j}$ where $i$ is an index denoting its layer (output layer is 0, second-to-last 1, etc.) and  $j$ is some index that differentiates the neuron from other ones in the given layer. Let's call the weights of $\mathcal{N}_{i, j}$ $w_{i, j}$, the weighted sum of inputs of that neuron $z_{i,j}$ and its activation function $act_{i,j}$.
Returning to our example of deriving some loss function with respect to the weights of a neuron in the second-to-last layer we get 
$$ \frac{ \partial \mathcal{L}}{\partial w_{1,j_0}} = \frac{ \partial \mathcal{L}}{\partial act_{0,0}} \cdot \frac{ \partial act_{0,0}}{\partial z_{0,0}} \cdot \frac{ \partial z_{0, 0}}{\partial act_{1,j_0}}\cdot \frac{\partial act_{1,j_0}}{ \partial w_{1,j_0}}
$$

Automatic derivation now decomposes these derivatives into its base-parts, fills in the derivations and evaluates. For example, because $z_{0,0}$ is a sum, autodiff would replace this with the sum of the derivatives. This will be a sum of derivatives of activation functions which can be further derived or, if they are independent of the weights we are interested in, can be set to 0. 

Let us code an algorithm that illustrates: 

In [8]:
[].__len__()

0

In [9]:
import numpy as np
a = np.array([1,2,3])
f = lambda x: x+1
f_vec = np.vectorize(f)

f_vec(a)

array([2, 3, 4])

In [10]:
import numpy as np
class expression: 
    def __init__(self, operator, left, right) -> None:
        self.operator = operator
        self.left = left
        self.right = right

    def evaluate(self):
        return eval(self.left+self.operator+self.right)
arr = expression("+", "23", "8"), expression("+", "23", "8"), expression("+", "23", "8"), expression("+", "23", "8")

getvals= np.vectorize(expression.evaluate)
getvals(arr)

array([31, 31, 31, 31])

In [16]:
class count:
    classvar = 0
    def __init__(self) -> None:
        self.ID = count.classvar 
        count.classvar += 1
    giveID = lambda self: self.ID
    give_arr = np.vectorize(giveID)

a, b, c = count(), count(), count()

for i in a,b,c: 
    print(count.give_arr([a,b,c]))
        

[0 1 2]
[0 1 2]
[0 1 2]


In [27]:
# @ Author: Jakob Abou Zeid
# @ email: jakob.abouzeid@gmail.com


import typing
from collections.abc import Callable
import numpy as np

class Neuron:
    next_neuron_ID = 0
    def __init__(self, in_list: typing.List['Neuron'], activation: Callable) -> None:
        self.activation = activation
        self.parameters = np.random.rand(in_list.__len__()+1,1)
        self.in_list = in_list
        self.out_value = None
        self.neuron_ID = Neuron.next_neuron_ID
        Neuron.next_neuron_ID +=1

    def give_output(self: 'Neuron'):
        if self.out_value is None:
            self.activate()
        return self.out_value

    def get_inputs(self):
        # We reshape the input we get rather than the output we deliver. 
        # This ensures that the input layer can be initialized with rows or columns or 1-d arrays. Handy!
        return np.hstack([in_neuron.give_output().reshape(-1,1) for in_neuron in self.in_list])

    def activate(self):
        inputs = self.get_inputs()
        z = np.hstack([inputs, np.ones((inputs.shape[0],1))]).dot(self.parameters)
        z = np.sum(z,axis=1)
        self.out_value = self.activation(z)
        return (self.out_value)

    def reset(self):
        self.out_value = None
        for child in self.in_list:
            child.reset()
    


test = [Neuron([], lambda x:x), Neuron([], lambda x:x)]
for neuron in test:
    neuron.out_value = np.arange(10).reshape(-1)

parent = Neuron(test, lambda x:x)
parent.parameters = 1
parent.activate()


array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19.])

In [32]:
None or "A"

'A'

In [31]:
def print_all_IDs(root: Neuron):
    print(root.neuron_ID)
    for neuron in root.in_list:
        print_all_IDs(neuron)

print_all_IDs(parent)

2
0
1


In [ ]:
# Make an input layer of 2 neurons:
num_input_neurons = 2
in_layer = [Neuron([], lambda x:x) for i in range(num_input_neurons)]

for neuron, val in zip(in_layer, [2,3]):
    neuron.out_value = val

out_neuron = Neuron(in_layer, lambda x:x)
out_neuron.parameters = 1

out_neuron.activate()

In [43]:
arr1 = np.arange(4).reshape(-1,1)
arr2 = np.arange(4).reshape(-1,1)

np.hstack([arr1, arr2])

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3]])

In [62]:
import math

class Var:
    nextID = 'a'
    def __init__(self, value, children=None, coeff = None):
        self.value = value
        self.children = children or []
        self.grad = 0
        self.coeff = coeff
        self.ID = Var.nextID
        Var.nextID = chr(ord(Var.nextID)+1)

    def __add__(self, other):
        return Var(self.value + other.value, [self, other], dict([(self.ID, 1), (other.ID, 1)]))

    def __mul__(self, other):
        return Var(self.value * other.value, [self, other],  dict([(self.ID, other.value), (other.ID, self.value)]))

    def sin(self):
        return Var(math.sin(self.value),[self], {self.ID: math.cos(self.value)})

    def calc_grad(self, grad=1):
        self.grad += grad
        for child in self.children:
            coeff = self.coeff[child.ID]
            print(f"deriving {self.ID} wrt {child.ID} (Value {child.value}): ", f"grad = {self.grad}, coeff= {coeff}")
            child.calc_grad(grad * coeff)

# Example: f(x, y) = x * y + sin(x)
x = Var(2)
y = Var(3)
f = x * y 
f = f + x.sin()

# Calculation of partial derivatives
f.calc_grad()

print("f =", f.value)
print("∂f/∂x =", x.grad, f"[∂{f.ID}/∂{x.ID}]")
print("∂f/∂y =", y.grad)

deriving e wrt c (Value 6):  grad = 1, coeff= 1
deriving c wrt a (Value 2):  grad = 1, coeff= 3
deriving c wrt b (Value 3):  grad = 1, coeff= 2
deriving e wrt d (Value 0.9092974268256817):  grad = 1, coeff= 1
deriving d wrt a (Value 2):  grad = 1, coeff= -0.4161468365471424
f = 6.909297426825682
∂f/∂x = 2.5838531634528574 [∂e/∂a]
∂f/∂y = 2


In [53]:
def inOrder(root: Var, indents = 0):
    print("\t"*indents, root.ID, ":", root.value)
    for child in root.children:
        inOrder(child, indents+1)



In [54]:
inOrder(f)

 e : 6.909297426825682
	 c : 6
		 a : 2
		 b : 3
	 d : 0.9092974268256817
		 a : 2


In [ ]:
# @ Author: Jakob Abou Zeid
# @ email: jakob.abouzeid@gmail.com
import typing
from collections.abc import Callable
import numpy as np

class Neuron:
    next_neuron_ID = 0
    def __init__(self, in_list: typing.List['Neuron'], activation: Callable) -> None:
        self.activation = activation
        self.parameters = np.random.rand(in_list.__len__()+1,1)
        self.in_list = in_list
        self.out_value = None
        self.neuron_ID = Neuron.next_neuron_ID
        self.parents = []
        Neuron.next_neuron_ID +=1

    def give_output(self: 'Neuron') -> float:
        if self.out_value is None:
            self.activate()
        return self.out_value

    def get_inputs(self):
        # We reshape the input we get rather than the output we deliver. 
        # This ensures that the input layer can be initialized with rows or columns or 1-d arrays. Handy!
        return np.hstack([in_neuron.give_output().reshape(-1,1) for in_neuron in self.in_list])

    def activate(self):
        inputs = self.get_inputs()
        z = np.hstack([inputs, np.ones((inputs.shape[0],1))]).dot(self.parameters)
        z = np.sum(z,axis=1)
        self.out_value = self.activation(z)
        return (self.out_value)

    def reset(self):
        self.out_value = None
        for child in self.in_list:
            child.reset()
    
    def set_children_backlink(self):
        for child in self.in_list:
            child.parents.append(self)
            child.set_children_backlink()
        

class Net:
    def __init__(self, root: Neuron):
        self.root = root
        root.set_children_backlink()
    def train():
        pass
    
    
            
    
    


test = [Neuron([], None), Neuron([], None)]
for neuron in test:
    neuron.out_value = np.arange(10).reshape(-1)

parent = Neuron(test, lambda x:x, [lambda : 1, lambda : 1])
parent.parameters = 1
parent.activate()
